In [1445]:
import pandas as pd

In [1446]:
import io

In [1447]:
import os

In [1448]:
import re

In [1449]:
pip install chardet

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: /jupyter/.kernels/apache-beam-2.46.0/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1450]:
import chardet

In [1451]:
import apache_beam as beam
from apache_beam.io import jdbc ,ReadFromText,WriteToText


In [1452]:
import psycopg2

In [1453]:

from apache_beam.io.gcp.gcsfilesystem import GCSFileSystem
from google.cloud import storage


In [1454]:
pip install openpyxl

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: /jupyter/.kernels/apache-beam-2.46.0/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1455]:
pip install google-cloud-storage

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: /jupyter/.kernels/apache-beam-2.46.0/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1456]:
pip install psycopg2-binary

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: /jupyter/.kernels/apache-beam-2.46.0/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1457]:
items=[ 'id',
    'name',
    'domain',
    'year_of_experience',
    'education',
    'dob',
    'skill set',
    'expected_salary',
    'email',
    'mobilenumber',
    'country code',
    'availability date',
    'gender',
    'country',
    'created at',
    'updated at',
    'deleted at',
    'notice_period',
    'city/town',
    'state',
    'address',
    'certificate training',
    'current salary',
    'previous company',
    'sub_category_id',
       'verified'
       
]

In [1458]:
project_id = 'recruitment-assistance-backend'
bucket_name = 'resume_transform'
file_pattern = 'gs://{}/my_dataframe*.csv'.format(bucket_name)
file_patterntxt = 'gs://{}/*.txt'.format(bucket_name)
connection_name = 'recruitment-assistance-backend:us-central1:recruitment-assistance'
databasex='recruitment-assistance'
database_name = 'recruitment-db'
username = 'recruitment-dev'
password = "P@ssword"
table_name = 'sourcing_sourcecandidate'


options = {
    'project': project_id,
    'region': 'us-central1',
    'runner': 'DataflowRunner',
    'job_name': 'load-csv-to-postgres',
    'temp_location': 'gs://{}/temp'.format(bucket_name),
    'staging_location': 'gs://{}/staging'.format(bucket_name),
    'requirements_file' : 'requirements.txt'
}

In [1459]:
dfbucket = client.get_bucket(bucket_name)
dfblobs = list(bucket.list_blobs())

In [1460]:
dfpattern = r'mydataframe.*\.csv' 
# Iterate over the files and delete the ones that match the pattern 
for blob in dfblobs:
    if blob.name.startswith('my_dataframe'):
        blob.delete()


In [1461]:
src_bucket_name = bucket_name
src_folder_name_fb = "source_facebook/"
src_folder_name_gf = "source_google/"
dest_bucket_name = bucket_name
dest_folder_name = "Loaded/"


In [1462]:
client = storage.Client()
# Create a list to store the dataframes of all Excel files
all_dataframes = []

# Get a list of all the blobs (files) in the source folder of the source bucket
src_bucket = client.bucket(src_bucket_name)
src_folder = src_bucket.blob(src_folder_name_fb)
all_blobs = list(src_bucket.list_blobs(prefix=src_folder.name))


In [1463]:
all_blobs

[<Blob: resume_transform, source_facebook/, 1683135186429704>,
 <Blob: resume_transform, source_facebook/cro dev data.xlsx, 1684605285867196>]

In [1464]:
all_dataframes_gf = []

src_folder = src_bucket.blob(src_folder_name_gf)
all_blobs_gf = list(src_bucket.list_blobs(prefix=src_folder.name))

In [1465]:

if len(all_blobs) > 1:
    # Loop through each blob (file) in the list
    for blob in all_blobs:
        # Check if the blob is an Excel file
        if blob.name.endswith(".xlsx") or blob.name.endswith(".xls"):
            # Load the Excel file into a Pandas dataframe
            excel_data = blob.download_as_string()
            df = pd.read_excel(excel_data, engine="openpyxl")
            all_dataframes.append(df)

            
            
            # Move the file to the destination folder of the destination bucket
            src_bucket.rename_blob(blob, dest_folder_name+all_blobs[1].name.split("/")[-1])
            #src_bucket.copy_blob(blob.name, dest_bucket_name, dest_folder_name + blob.name.split("/")[-1])
            #blob.delete()
        if blob.name.endswith(".csv"):
            byte_data = blob.download_as_bytes()
            result = chardet.detect(byte_data)
            encoding = result['encoding']
            print(encoding)
            if encoding == 'UTF-8' or encoding == 'UTF-16':
                # blob.download_to_filename(temp)
                UTF = byte_data.decode('utf-16')
                df = pd.read_csv(io.StringIO(UTF),delimiter = '\t')
                all_dataframes.append(df)
            else:
                df = pd.read_csv(io.BytesIO(byte_data))


In [1466]:
len(all_blobs)

2

In [1467]:


# Loop through each blob (file) in the list
for blob in all_blobs_gf:
    # Check if the blob is an Excel file
    if blob.name.endswith(".xlsx"):
        # Load the Excel file into a Pandas dataframe
        excel_data = blob.download_as_string()
        df = pd.read_excel(excel_data, engine="openpyxl")
        all_dataframes_gf.append(df)
        
        # Move the file to the destination folder of the destination bucket
        src_bucket.rename_blob(blob, dest_folder_name+all_blobs_gf[1].name.split("/")[-1])
        #src_bucket.copy_blob(blob.name, dest_bucket_name, dest_folder_name + blob.name.split("/")[-1])
        #blob.delete()
  

In [1468]:
all_dataframes

[          form_name  is_organic platform                         email  \
 0   Soho Specialist       False       fb  money.chocolateboy@gmail.com   
 1   Soho Specialist       False       fb   narendrasamriya28@gmail.com   
 2   Soho Specialist       False       fb    mahendrasiyag421@gmail.com   
 3   Soho Specialist       False       fb           pk9169283@gmail.com   
 4   Soho Specialist       False       fb      guptarahul0334@gmail.com   
 5   Soho Specialist       False       fb        nitinraj2601@gmail.com   
 6   Soho Specialist       False       fb   bharatazadchd1992@gmail.com   
 7   Soho Specialist       False       fb    thakurpankaj1236@gmail.com   
 8   Soho Specialist       False       fb     raoshab.parveen@gmail.com   
 9   Soho Specialist       False       fb   jatinder.singh@vodafone.com   
 10  Soho Specialist       False       fb  gulshanjairamani13@gmail.com   
 11  Soho Specialist       False       fb      vickysinghal11@gmail.com   
 12  Soho Specialist     

In [1469]:
final_df_fb = pd.concat(all_dataframes, ignore_index=True)

In [1470]:
final_df_fb.columns = final_df_fb.columns.str.lower()

In [1471]:
final_df_fb['phone_number'] = final_df_fb['phone_number'].replace(r'[^0-9.]', '',regex = True)


In [1472]:
final_df_fb

,form_name,is_organic,platform,email,full_name,phone_number,street_address,city,state,post_code,gender,date_of_birth,job_title,company_name,sub_category_id
0,Soho Specialist,False,fb,money.chocolateboy@gmail.com,Gurpreet Singh,919988330399,amritsar,amritsar,punjab,143001,male,12/10/1990,teacher,Bajaj Allianz,4
1,Soho Specialist,False,fb,narendrasamriya28@gmail.com,Narendra Samriya,917891056338,kota,Kota,rajasthan,324005,male,05/12/1996,telecom,"Kota, Rajasthan",4
2,Soho Specialist,False,fb,mahendrasiyag421@gmail.com,mahi choudhary,919079917494,01 asanda bhopalgarh,Jodhpur,rajasthan,342901,male,07/10/2000,store,Mi Hitesh Panchal,4
3,Soho Specialist,False,fb,pk9169283@gmail.com,Prakash Kumar paswan,919708870511,vill-Nilo rukundpur post Davaich police statio...,patepur,Bihar,843110,male,03/27/1997,warranty adminstrator,Mahindra and Mahindra,4
4,Soho Specialist,False,fb,guptarahul0334@gmail.com,Rahul Gupta,919758547853,milak,Rampur,up,243701,male,02/02/1992,0,Business oooo,4
5,Soho Specialist,False,fb,nitinraj2601@gmail.com,Nitin raj singh,917906639219,Aligarh,Aligarh,Uttar Pradesh,202001,male,01/26/1995,Business Analyst in Information technology,CRM,4
6,Soho Specialist,False,fb,bharatazadchd1992@gmail.com,vedprakash,918433080719,Sambhal,Chandausi,U.P,244410,male,07/01/1992,part time,3 year experience,4
7,Soho Specialist,False,fb,thakurpankaj1236@gmail.com,पंकज ठाकुर,918476887379,चंडौस,चंडौस अलीगढ़,uttar prdesh,202155,male,07/15/1997,tsm,Airtel,4
8,Soho Specialist,False,fb,raoshab.parveen@gmail.com,Parveen Rao,919728754534,Rewari,Rewari,HARYANA,123303,male,09/04/1988,SR.CRO,BlackBuck,4
9,Soho Specialist,False,fb,jatinder.singh@vodafone.com,Jatinder Singh,919646230134,friends colony jodh nagar sultanwind road asr,Amritsar,punjab,143001,male,01/20/1989,Assistant Professor,Bajaj Allianz General Insurance,4


In [1473]:
if len(all_blobs_gf) > 1:
    final_df_gf = pd.concat(all_dataframes_gf, ignore_index=True)    

In [1474]:
all_blobs

[<Blob: resume_transform, source_facebook/, 1683135186429704>,
 <Blob: resume_transform, source_facebook/cro dev data.xlsx, 1684605285867196>]

In [1475]:
final_df_gf.columns = final_df_gf.columns.str.lower()

In [1476]:
data_fb= final_df_fb
data_google=final_df_gf

In [1477]:
# Map the columns to the desired names
data_fb = data_fb.rename(columns={
    'what_is_your_expected_salary?': 'expected_salary',
    'years_of_experience': 'year_of_experience',
    'full_name': 'name',
    'phone_number': 'mobilenumber',
    'city' : 'city/town',
    'company_name' : 'previous company',
    'street_address' : 'address',
    'what_is_your_qualification?' : 'education',
    'what_is_your_expected_salary?' : 'expected_salary',
    'some_of_your_skill_sets?' : 'skill set',
    'what_is_your_notice_period?' : 'notice_period'
    
 
    
})
# Map the columns to the desired names
data_google = data_google.rename(columns={

    'What is your expected salary?': 'expected_salary',
    'Years Of Experience (in field of Sales and Marketing)': 'year_of_experience',
    'Name': 'name',
    'Contact Number (WhatsApp)': 'mobilenumber',
    'Education':'education',
    'State' : 'state'
    
})

In [1478]:
for i in items:
    if i not in data_google:
        data_google[i]=''
    if i not in data_fb:
        data_fb[i]=''


In [1479]:
data_google['id']=''

In [1480]:
data_google = data_google[[
    'id',
    'name',
    'domain',
    'year_of_experience',
    'education',
    'dob',
    'skill set',
    'expected_salary',
    'email',
    'mobilenumber',
    'country code',
    'availability date',
    'gender',
    'country',
    'created at',
    'updated at',
    'deleted at',
    'notice_period',
    'city/town',
    'state',
    'address',
    'certificate training',
    'current salary',
    'previous company',
    'sub_category_id',
    'verified'
]]
# Reorder the columns
data_fb = data_fb[[
    'id',
    'name',
    'domain',
    'year_of_experience',
    'education',
    'dob',
    'skill set',
    'expected_salary',
    'email',
    'mobilenumber',
    'country code',
    'availability date',
    'gender',
    'country',
    'created at',
    'updated at',
    'deleted at',
    'notice_period',
    'city/town',
    'state',
    'address',
    'certificate training',
    'current salary',
    'previous company',
    'sub_category_id',
    'verified'
]]


In [1481]:
df_union = pd.concat([data_fb, data_google])

In [1482]:
df_union.reset_index(drop=True, inplace=True)
df_union =df_union.drop(columns=['id'])

In [1483]:
df_union.replace(r'^\s*$', 'NULL', regex=True, inplace=True)
df_union.replace(r',', '', regex=True, inplace=True)
df_union['verified'] = df_union['verified'].str.replace('NULL','False',regex = True)
df_union['expected_salary'] = df_union['expected_salary'].replace(r'[^0-9]', '',regex = True)
df_union['country code'] = df_union['country code'].str.replace('NULL','91',regex = True)

In [1484]:
df_union = df_union[df_union['email'] != 'darshnashrivastava08@gmail.com']

In [1485]:
df_union

,name,domain,year_of_experience,education,dob,skill set,expected_salary,email,mobilenumber,country code,...,deleted at,notice_period,city/town,state,address,certificate training,current salary,previous company,sub_category_id,verified
0,Gurpreet Singh,NULL,NULL,NULL,NULL,NULL,,money.chocolateboy@gmail.com,919988330399,91,...,NULL,NULL,amritsar,punjab,amritsar,NULL,NULL,Bajaj Allianz,4,False
1,Narendra Samriya,NULL,NULL,NULL,NULL,NULL,,narendrasamriya28@gmail.com,917891056338,91,...,NULL,NULL,Kota,rajasthan,kota,NULL,NULL,Kota Rajasthan,4,False
2,mahi choudhary,NULL,NULL,NULL,NULL,NULL,,mahendrasiyag421@gmail.com,919079917494,91,...,NULL,NULL,Jodhpur,rajasthan,01 asanda bhopalgarh,NULL,NULL,Mi Hitesh Panchal,4,False
3,Prakash Kumar paswan,NULL,NULL,NULL,NULL,NULL,,pk9169283@gmail.com,919708870511,91,...,NULL,NULL,patepur,Bihar,vill-Nilo rukundpur post Davaich police statio...,NULL,NULL,Mahindra and Mahindra,4,False
4,Rahul Gupta,NULL,NULL,NULL,NULL,NULL,,guptarahul0334@gmail.com,919758547853,91,...,NULL,NULL,Rampur,up,milak,NULL,NULL,Business oooo,4,False
5,Nitin raj singh,NULL,NULL,NULL,NULL,NULL,,nitinraj2601@gmail.com,917906639219,91,...,NULL,NULL,Aligarh,Uttar Pradesh,Aligarh,NULL,NULL,CRM,4,False
6,vedprakash,NULL,NULL,NULL,NULL,NULL,,bharatazadchd1992@gmail.com,918433080719,91,...,NULL,NULL,Chandausi,U.P,Sambhal,NULL,NULL,3 year experience,4,False
7,पंकज ठाकुर,NULL,NULL,NULL,NULL,NULL,,thakurpankaj1236@gmail.com,918476887379,91,...,NULL,NULL,चंडौस अलीगढ़,uttar prdesh,चंडौस,NULL,NULL,Airtel,4,False
8,Parveen Rao,NULL,NULL,NULL,NULL,NULL,,raoshab.parveen@gmail.com,919728754534,91,...,NULL,NULL,Rewari,HARYANA,Rewari,NULL,NULL,BlackBuck,4,False
9,Jatinder Singh,NULL,NULL,NULL,NULL,NULL,,jatinder.singh@vodafone.com,919646230134,91,...,NULL,NULL,Amritsar,punjab,friends colony jodh nagar sultanwind road asr,NULL,NULL,Bajaj Allianz General Insurance,4,False


In [1486]:
import datetime

time_now  = datetime.datetime.now().strftime('%m_%d_%Y_%H_%M_%S') 

print(time_now)

05_20_2023_17_55_26


In [1487]:
from datetime import datetime

In [1488]:
today_date = datetime.today().strftime('%m/%d/%y') 
df_union['availability date'].replace('NULL',str(today_date), inplace=True)

In [1489]:
df_union['created at'].replace('NULL',str(today_date + ' 00:00:00'), inplace=True)
df_union['updated at'].replace('NULL',str(today_date + ' 00:00:00'), inplace=True)
df_union['deleted at'].replace('NULL',str(today_date + ' 00:00:00'), inplace=True)
df_union['dob'].replace('NULL',str(today_date + ' 00:00:00'), inplace=True)

In [1490]:
file_path = 'my_dataframe'+time_now+'.csv'
# Save the DataFrame to a CSV file
df_union.to_csv(file_path, index=False,header=False)

# Upload the CSV file to GCS
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_path)
blob.upload_from_filename(file_path)

In [1491]:
dfls =df_union.values.tolist()

load1 =['Aâ¤ï¸R', 'NULL', 'NULL', 'NULL', '05/20/23 00:00:00', 'NULL', '', 'anjalipandeyt@gmail.com', '918299860282', '91', '05/20/23', 'Female', 'NULL', '05/20/23 00:00:00', '05/20/23 00:00:00', '05/20/23 00:00:00', 'NULL', 'Lucknow', 'Uttar Pradesh', 'Lucknow', 'NULL', 'NULL', 'Airtel', '10', 'False']

In [1498]:
'''load1[8]'''

'load1[8]'

In [1499]:
'''load_to_postgres(load1, connection_name, database_name, username, password, table_name)'''


'load_to_postgres(load1, connection_name, database_name, username, password, table_name)'

In [1494]:
def load_to_postgres(batch, connection_name, database_name, username, password, table_name):
    import psycopg2
    import logging
    connection = psycopg2.connect(
        host='10.1.224.3',
        port= 5432,
        database=database_name,
        user=username,
        password=password
    )

    cursor = connection.cursor()
    cursor.execute(f"SELECT * FROM {table_name} WHERE mobile_number = %s", (str(batch[8]),))
    
    existing_row = cursor.fetchone()
    idquery="SELECT setval('sourcing_sourcecandidate_id_seq', (SELECT MAX(id) FROM sourcing_sourcecandidate))"
    cursor.execute(idquery)
    batch[5]=[batch[5]]
    if existing_row:
        query = "UPDATE {0} SET candidate_name= %s,domain= %s,total_experience= %s,highest_education= %s,\
        date_of_birth= %s,skill_set=%s,expected_salary= %s,email_address= %s,mobile_number= %s,\
        country_code= %s,availability_date= %s,gender= %s,\
        country= %s,created_at= %s,updated_at= %s,deleted_at= %s,notice_period= %s,\
        city= %s,state= %s,address= %s,certificate_training= %s,current_salary= %s,previous_company= %s\
        ,sub_category_id= %s,verified= %s\
        WHERE mobile_number = '{1}' ".format(table_name,str(batch[8]))
        #query= query.replace("array{'", "array{\"") 
        #query= query.replace("'}", "\"}")
        cursor.execute(query, list(batch))
        print(query)
    else:
        query = "INSERT INTO {0} (candidate_name,domain,total_experience,highest_education,date_of_birth,skill_set,expected_salary,email_address,mobile_number,\
        country_code,availability_date,gender,country,created_at,updated_at,deleted_at,notice_period,city,\
        state,address,certificate_training,current_salary,previous_company,sub_category_id,verified) \
        VALUES (%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)".format(table_name)
        #query= query.replace("array{'", "array{\"") 
        #query= query.replace("'}", "\"}")
        logging.info('%s', query)
        logging.info('%s', list(batch))
        cursor.execute(query, list(batch))
        print(query)
        #if cu
    #for line in batch:
        # Split the line into fields
        #fields = line.split(',')

        # Check if the line already exists in the table
        #query = "SELECT COUNT(*) FROM {} WHERE id = %s".format(table_name)
        #cursor.execute(query, (batch[0],))

        #if cursor.fetchone()[0] == 0:
            # Insert the line into the table



    connection.commit()
    cursor.close()
    connection.close()


In [1495]:
for i in range(0,len(dfls)-1):
    load_to_postgres(dfls[i], connection_name, database_name, username, password, table_name)


INSERT INTO sourcing_sourcecandidate (candidate_name,domain,total_experience,highest_education,date_of_birth,skill_set,expected_salary,email_address,mobile_number,        country_code,availability_date,gender,country,created_at,updated_at,deleted_at,notice_period,city,        state,address,certificate_training,current_salary,previous_company,sub_category_id,verified)         VALUES (%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
INSERT INTO sourcing_sourcecandidate (candidate_name,domain,total_experience,highest_education,date_of_birth,skill_set,expected_salary,email_address,mobile_number,        country_code,availability_date,gender,country,created_at,updated_at,deleted_at,notice_period,city,        state,address,certificate_training,current_salary,previous_company,sub_category_id,verified)         VALUES (%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
INSERT INTO sourcing_sourcecandidate (candidate_name,dom

'''
class MoveFile(beam.DoFn):
    def __init__(self, source_bucket, source_prefix, dest_bucket, dest_prefix):
        self.source_bucket = source_bucket
        self.source_prefix = source_prefix
        self.dest_bucket = dest_bucket
        self.dest_prefix = dest_prefix
        self.gcs = GCSFileSystem()

    def process(self, element):
        source_file = element.strip()
        source_path = f"gs://{self.source_bucket}/{self.source_prefix}/{source_file}"
        dest_path = f"gs://{self.dest_bucket}/{self.dest_prefix}/{source_file}"
        blob = storage.blob.Blob.from_string(source_path, self.gcs)
        self.gcs.rename(blob, dest_path)
        yield source_file'''

In [1496]:
'''def load_to_postgres(batch, connection_name, database_name, username, password, table_name):
    import psycopg2
    import logging
    connection = psycopg2.connect(
        host='172.24.32.3',
        port= 5432,
        database=database_name,
        user=username,
        password=password
    )

    cursor = connection.cursor()

    #for line in batch:
        # Split the line into fields
        #fields = line.split(',')

        # Check if the line already exists in the table
        #query = "SELECT COUNT(*) FROM {} WHERE id = %s".format(table_name)
        #cursor.execute(query, (batch[0],))
    cursor.execute(f"SELECT * FROM {table_name} WHERE id = %s", (batch[0],))
    existing_row = cursor.fetchone()
    if existing_row:
        cursor.execute(f"UPDATE {table_name} SET name = %s, age = %s WHERE id = %s", (row[1], row[2], row[0]))
    else:
        cursor.execute(f"INSERT INTO {table_name} (id, name, age) VALUES (%s, %s, %s)", row)
    
        #if cursor.fetchone()[0] == 0:
            # Insert the line into the table
    query = "INSERT INTO {} (app_name,click_event,color,country,customer_id,date,device_name,device_type,discount,email,\
    ip_address,ip_browser,is_ordered_flag,item_category,item_name,payment_method,\
    router_mac_address) VALUES (%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s,%s)".format(table_name)
    logging.info('%s', query)
    cursor.execute(query, list(batch))

    connection.commit()
    cursor.close()
    connection.close()
'''

'def load_to_postgres(batch, connection_name, database_name, username, password, table_name):\n    import psycopg2\n    import logging\n    connection = psycopg2.connect(\n        host=\'172.24.32.3\',\n        port= 5432,\n        database=database_name,\n        user=username,\n        password=password\n    )\n\n    cursor = connection.cursor()\n\n    #for line in batch:\n        # Split the line into fields\n        #fields = line.split(\',\')\n\n        # Check if the line already exists in the table\n        #query = "SELECT COUNT(*) FROM {} WHERE id = %s".format(table_name)\n        #cursor.execute(query, (batch[0],))\n    cursor.execute(f"SELECT * FROM {table_name} WHERE id = %s", (batch[0],))\n    existing_row = cursor.fetchone()\n    if existing_row:\n        cursor.execute(f"UPDATE {table_name} SET name = %s, age = %s WHERE id = %s", (row[1], row[2], row[0]))\n    else:\n        cursor.execute(f"INSERT INTO {table_name} (id, name, age) VALUES (%s, %s, %s)", row)\n    \n     


with beam.Pipeline(options=beam.pipeline.PipelineOptions.from_dictionary(options)) as p:
    # Read CSV file from Cloud Storage
    data = p | ReadFromText(file_pattern) 

    # Transform data as necessary
    transformed_data = data | beam.Map(lambda row: row.split(','))

    # Write data to batch files in Cloud Storage
    '''batch_files = transformed_data | WriteToBatchFiles(
        file_path_prefix='gs://{}/output'.format(bucket_name),
        shard_name_template='',
        batch_size=1000
    )'''
    # Write transformed data to text file
    out_blob = bucket.blob('output-00000-of-00001.txt')
    out_blob.delete()
    text_files = transformed_data | WriteToText(
        file_path_prefix='gs://{}/output'.format(bucket_name),
        file_name_suffix='.txt'
    )
    #datatxt = ReadFromText(file_patterntxt)

    # Read data from text file and load into PostgreSQL table
    _ = text_files |'LoadToPostgres' >> beam.ParDo(load_to_postgres, connection_name, database_name, username, password, table_name)
    # Move CSV file to archive folder
    #_ = data | beam.ParDo(MoveFile('myfiles-test', 'rut.csv', 'myfiles-test', 'archive'))

In [1497]:
'''python -m plsqlload (4).ipynb\
    --runner DataflowRunner \
    --project fine-program-384407 \
    --staging_location gs://myfiles-test/staging \
    --temp_location gs://myfiles-test/temp \
    --template_location gs://myfiles-test/templates/TEMPLATE_NAME \
    --region REGION'''

'python -m plsqlload (4).ipynb    --runner DataflowRunner     --project fine-program-384407     --staging_location gs://myfiles-test/staging     --temp_location gs://myfiles-test/temp     --template_location gs://myfiles-test/templates/TEMPLATE_NAME     --region REGION'